# Feature Engineering

This notebook is dedicated to creating features for our heart rate prediction model. We add a lot of features which are partly inspired by the cycing domain (like trainingload, 'threshold power', etc.), but also features which are more explorative in nature. The relation of these features with our target variable will be explored in a next notebook **3. Feature exploration and selection.ipynb**

First we recap some of the learings of our notebook **0. Data exploration.ipynb**

**Learnings from 0. Data exploration.ipynb**

We know that heart rate runs smoother than power (watts). We have also seen that heart rate is lagging an increase or decrease in power. Also we observed something which is called cardiac drift. This means that overtime heart rate tends to increase even if power stays constant. A lot of factors play a role here which cause heart rate to drift upwards (e.g. hydration, muscle recruitment, drop in stroke volume, etc.) which we broadly define as "fatigue". We also observe that heart rate goes up after consequtive intervals even though power remains rather constant during the intervals. This also shows we should take a time effect into account

This notebook is organised as follows:
* Read in data from our previous notebook **1. Data processing.ipynb**
* Cycling specific ride file derived features
* Data driven features
     * Cumulative moving averages
     * Lagged variables
     * Simple moving averages
     * Deltas on moving averages
     * Exponentially weighted moving averages
     * Rolling standard deviations
     * Extra possible related variables
* Cycling (training)meta data features
     * Cycling ride related features
     * Cycling (training)meta data features
     * (Cycling) (training)meta data indexed features
     
Note: We make extensive use of a function to reduce memory everytime new features are created to avoid memory issues

### Import libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 999

import pickle
import numpy as np
import math
import gc

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# import function to reduce memory during processing
from reduce_memory import reduce_mem_usage

# visualisations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# read in the file and preprocess

def read_file(filename):
    '''
    Returns a DataFrame for further processing

            Parameters:
                    filename: (str) pickle object name
                    
            Returns:
                    pd.DataFrame: pandas DataFrame for further processing
    '''        
    df = pd.read_pickle(filename)
    
    df['secs'] = df.groupby('filename').cumcount()+1
    
    df.sort_values(['filename', 'secs'], ascending=[True, True]).reset_index(drop=True, inplace=True)
    
    df = reduce_mem_usage(df)
    
    return df

df_raw = read_file('df_modelset_rider1.pickle')

Memory usage after optimization is: 154.81 MB
Decreased by 50.0%


## Cycling specific features which are in the ride files

We calculate two new features using the following formula:

Power = Torque x Rotation Speed 

We calculate these two new features. Rotation speed in this equation is in radians/second. Convert RPM to radians/second by multiplying 2 x Pi / 60 (= 0.1047). Torque is in Newton-Meters (Nm)

Example: rider is riding at 400 Watts at and riding with a cadence of 100 RPM, the rotation speed is (100 RPM) x (0.1047 rad/sec/RPM) = 10.47 rad/sec. The applied torque is then (400 Watts) / (10.47 rad/sec) = 38.20 Nm. This torque is applied to the cranks

In [3]:
# Before continuing we make a small correction to the watts and cadence. If watts is zero, cadence will be put to zero. 
# Also when cadence is zero watts will be put to zero

df_raw['cad'] = np.where((df_raw.watts==0) & (df_raw.cad>0), 0, df_raw.cad)
df_raw['watts'] = np.where((df_raw.watts>0) & (df_raw.cad==0), 0, df_raw.watts)

# calculate torque at crank and rotational speed

def add_simple_cycling_features(df):
    df['rotation_speed'] = np.round(((2*math.pi)/60)*df['cad'],2)
    df['torque'] = np.round(df['watts']/df['rotation_speed'],2)
    df['torque'] = df['torque'].fillna(0)
    df = reduce_mem_usage(df)
    return df

df_raw = add_simple_cycling_features(df_raw)

Memory usage after optimization is: 170.29 MB
Decreased by 21.4%


## Data driven features

In [4]:
# for variables with positive values we add some data transformations

transform_cols = ['watts','secs','cad','torque','rotation_speed']

for col in transform_cols:
    df_raw['log_' + col] = np.log1p(df_raw[col])
    df_raw['sqrt_' + col] = np.sqrt(df_raw[col])
    df_raw['exp_' + col] = df_raw[col].apply(lambda x: x**(2))
    df_raw['cub_' + col] = df_raw[col].apply(lambda x: x**(3))

df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 394.76 MB
Decreased by 37.8%


### Cumulative moving averages (CMA)

In [5]:
columns_cum = ['watts','cad','rotation_speed'] #note we did not includr torque due to overflow (the result can not be stored within the finite 32-bit float or 64-bit double)

for col in columns_cum:
    df_raw['cum_total_' + col] =  np.round((df_raw.groupby('filename')[col].apply(lambda x: x.cumsum())).astype(int),0)
    df_raw['cum_average_' + col] = np.round((df_raw.groupby('filename')[col].apply(lambda x: x.cumsum())/df_raw.secs),0)

df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 464.42 MB
Decreased by 13.0%


In [6]:
# for variables with positive values we add some data transformations

columns_cum = [col for col in df_raw.columns if 'cum_' in col]

for col in columns_cum:
    df_raw['log_' + col] = np.log1p(df_raw[col])
    df_raw['sqrt_' + col] = np.sqrt(df_raw[col])
    df_raw['exp_' + col] = df_raw[col].apply(lambda x: x**(2))

df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 665.67 MB
Decreased by 24.6%


### Lags

In [8]:
lags = range(5, 61, 5)
#leads = range(-60, -4, 5)

cols = ['watts','torque','cad','rotation_speed','slope']

for i in lags:
    for col in cols:
        shifted = df_raw.groupby("filename")[col].shift(i).to_frame()
        df_raw = pd.merge(df_raw, shifted.rename(columns=lambda x: str(i)+"s_lag_"+str(col)), left_index=True, right_index=True, how='left')

df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 1290.09 MB
Decreased by 30.2%


### Simple moving averages: each period has a similar weight (1/n) (SMA)

In [16]:
window_range = range(5, 61, 5)

cols = ['watts','torque','cad','rotation_speed','slope']

for i in window_range:
    for col in cols:
        sma = df_raw.groupby("filename")[col].rolling(window=i, adjust=False).mean().to_frame().droplevel(level=[0])
        df_raw = pd.merge(df_raw, sma.rename(columns=lambda x: str(i)+"s_sma_"+ str(col)), left_index=True, right_index=True, how='left')

# reduce memory usage
df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 1754.50 MB
Decreased by 44.3%


### Simple deltas for up to 10 seconds max

In [21]:
window_range = range(1, 11, 1)

cols = ['watts','torque','cad','rotation_speed']

for i in window_range:
    for col in cols:
        sma = (df_raw[col]-df_raw[col].shift(i).where(df_raw.filename.eq(df_raw.filename.shift(i)))).to_frame()
        df_raw = pd.merge(df_raw, sma.rename(columns=lambda x: str(i)+"s_delta_"+ str(col)), left_index=True, right_index=True, how='left')

# reduce memory usage
df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 2064.12 MB
Decreased by 18.4%


In [22]:
# Add rolling means up to 30 seconds to show rolling acceleration for '1s_delta_watts' from 5 to 30 sec
window_range = range(5, 31, 5)

cols = ['1s_delta_watts','1s_delta_torque','1s_delta_rotation_speed','1s_delta_cad']

for i in window_range:
    for col in cols:
        sma = df_raw.groupby("filename")[col].rolling(window=i, adjust=False).mean().to_frame().droplevel(level=[0])
        df_raw = pd.merge(df_raw, sma.rename(columns=lambda x: str(i)+"s_"+ str(col)+"_mean"), left_index=True, right_index=True, how='left')

# reduce memory usage
df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 2249.88 MB
Decreased by 19.9%


### Exponential moving averages: last period has 2/n+1 (alpha) importance, previous period has 1 - 2/n+1 importance (1-alpha)

In [23]:
window_range = range(5, 61, 5)

cols = ['watts','torque','cad','rotation_speed','slope']

for i in window_range:
    for col in cols:
        ewma = df_raw.groupby('filename')[col].apply(lambda x: x.ewm(span=i, adjust=False).mean()).to_frame()
        df_raw = pd.merge(df_raw, ewma.rename(columns=lambda x: str(i)+"s_ewma_"+ str(col)), left_index=True, right_index=True, how='left')
    
## reduce memory usage
df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 2714.30 MB
Decreased by 33.9%


### SMA on standard deviation of watts

In [24]:
window_range = range(5, 61, 5)

for i in window_range:
    
    stds = df_raw.groupby("filename").watts.rolling(window=i, adjust=False).std().to_frame().droplevel(level=[0])
    df_raw = pd.merge(df_raw, stds.rename(columns=lambda x: str(i)+"s_std_watts"), left_index=True, right_index=True, how='left')

# reduce memory usage
df_raw = reduce_mem_usage(df_raw)
#ewstd

Memory usage after optimization is: 2807.19 MB
Decreased by 9.0%


### Additional SMA variable for zero watts

In [25]:
df_raw['dummy_zero_watts'] = np.where(df_raw.watts==0,1,0)

window_range = range(5, 61, 5)

for i in window_range:
    
    sma = df_raw.groupby("filename").dummy_zero_watts.rolling(window=i, adjust=False).mean().to_frame().droplevel(level=[0])
    df_raw = pd.merge(df_raw, sma.rename(columns=lambda x: str(i)+"s_sma_dummy_watts"), left_index=True, right_index=True, how='left')

    # reduce memory usage
df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 2903.94 MB
Decreased by 9.1%


In [26]:
# free memory

del ewma, shifted, sma, stds
gc.collect();

## Cycling (training)meta data features

In this part of the notebook we add cycling specific features

First we calculate two features ourselves based on the second by second data:

* Skiba XPower
* NP (Normalized Power)

Validation of the calculation can be found in notebook **Additional Replicating xPower Python code.ipynb**

Moreover we add meta level features related to the athletes ability to produce power, aggregate ride metrics and training stress/load metrics. References for these variables can be found in the literature section of notebook **0. Data exploration.ipynb**

### Cycling ride related features

In [27]:
# We fill the zero values in our 25s SMA so we can easily define our first valid rolling 25s value starting point. Since these NaNs only appear in the first 25 secs of the files this is not a problem
# Inspired and modified from: https://stackoverflow.com/questions/36923494/pandas-dataframe-use-previous-row-value-for-complicated-if-conditions-to-deter

start_time = datetime.now()

df_part = df_raw[['filename','secs','25s_sma_watts','30s_sma_watts']]

df_part['25s_sma_watts_fill'] =  df_part['25s_sma_watts'].fillna(0)
df_part['30s_sma_watts_fill'] =  df_part['30s_sma_watts'].fillna(0)

# define period for calculating xpower. This is 25 secs
n = 25

def apply_func_decorator(func):
    prev_row = {}
    def wrapper(curr_row, **kwargs):
        val = func(curr_row, prev_row)
        prev_row.update(curr_row)
        prev_row['xPower_EMA'] = val
        return val
    return wrapper

@apply_func_decorator
def running_total(curr_row, prev_row):
    return np.where(prev_row.get('25s_sma_watts_fill')==0, curr_row['25s_sma_watts_fill'], np.round((curr_row['25s_sma_watts_fill'] - prev_row.get('xPower_EMA', 0))*(2/(n+1)) + prev_row.get('xPower_EMA', 0),2))

df_raw_1 = pd.DataFrame()

for i in df_part.filename.unique():
    df_init = df_part.loc[df_part.filename==i]
    df_init['xPower_EMA'] = df_init.apply(running_total, axis=1)
    df_raw_1 = df_raw_1.append(df_init, ignore_index=True)
    
df_raw_1.drop(columns=['25s_sma_watts','25s_sma_watts_fill'], inplace=True)

print('Time processing', datetime.now()-start_time)

# define period for calculating NP. This is 30 secs
n = 30

def apply_func_decorator(func):
    prev_row = {}
    def wrapper(curr_row, **kwargs):
        val = func(curr_row, prev_row)
        prev_row.update(curr_row)
        prev_row['NP_EMA'] = val
        return val
    return wrapper

@apply_func_decorator
def running_total(curr_row, prev_row):
    return np.where(prev_row.get('30s_sma_watts_fill')==0, curr_row['30s_sma_watts_fill'], np.round((curr_row['30s_sma_watts_fill'] - prev_row.get('NP_EMA', 0))*(2/(n+1)) + prev_row.get('NP_EMA', 0),2))

df_raw_2 = pd.DataFrame()

for i in df_part.filename.unique():
    df_init = df_part.loc[df_part.filename==i]
    df_init['NP_EMA'] = df_init.apply(running_total, axis=1)
    df_raw_2 = df_raw_2.append(df_init, ignore_index=True)

df_raw_2.drop(columns=['30s_sma_watts','30s_sma_watts_fill'], inplace=True)    
    
print('Time processing', datetime.now()-start_time)

Time processing 0:08:36.683549
Time processing 0:17:09.942277


In [28]:
# merge features to one another before we merge it as one file to df_raw

df_cycling_EMA = pd.merge(df_raw_1[['filename','secs','xPower_EMA']], df_raw_2[['filename','secs','NP_EMA']], on =['filename','secs'], how = 'left')

# correction on the first row of each file since for the first row of a new file the value was calculated using the previous file
df_cycling_EMA.loc[df_cycling_EMA.groupby('filename')['NP_EMA'].head(1).index, 'NP_EMA'] = 0
df_cycling_EMA.loc[df_cycling_EMA.groupby('filename')['xPower_EMA'].head(1).index, 'xPower_EMA'] = 0

# merge features to df_raw
df_raw = pd.merge(df_raw, df_cycling_EMA, on =['filename','secs'], how = 'left')

# garbage collect
del df_raw_1, df_raw_2, df_init, df_part, df_cycling_EMA
gc.collect()

# reduce memory usage
df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 2759.42 MB
Decreased by 1.7%


### Cycling (training)meta data features

We add two files. (1) One that is related to training load / CP and (2) one that is related to ride summary statistics. Note: These files were created using notebook **0. Data exploration.ipynb**

In [31]:
# Add training load information from PMC

def read_file_PMC():
    df = pd.read_pickle('df_PMC_CP.pkl')
    df.reset_index(inplace=True)
    return df

df_PMC_CP = read_file_PMC()

In [32]:
# make date format column in df_raw to merge with df_PMC data

df_raw['Date'] = df_raw['date'].astype(str).apply(lambda x: x.split('-')[1]) + "/" + df_raw['date'].astype(str).apply(lambda x: x.split('-')[2]) + "/" + df_raw['date'].astype(str).apply(lambda x: x.split('-')[0])

# merge the PMC data to df_raw
df_raw = pd.merge(df_raw, df_PMC_CP, how='left', on=['Date'])

# reduce memory usage
df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 2825.21 MB
Decreased by 6.0%


In [33]:
# read in summary statistics

def read_file_activities_summary():
    df = pd.read_pickle('df_summary_statistics.pkl')
    return df

df_summary_statistics = read_file_activities_summary()

# merge the summary statistics data to df_raw
df_raw = pd.merge(df_raw, df_summary_statistics, how='left', on=['date','filename'])

# reduce memory usage
df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 2960.67 MB
Decreased by 7.8%


### (Cycling) (training)meta data indexed features

Here we create features by scaling the wattage derived features by the rider's "threshold" for that specific period based on CP Ext. Notice the quotes. This is because there are a multitude of definitions and ways to derive this. We chose to use CP Ext, but other metrics like "FTP" could also be used to scale the data 

In [34]:
# First we specifically scale the cumulative variables and in addition create same derived variables as before on raw wattages

df_raw['CP_scaled_cum_average_watts'] = np.where((df_raw.cum_average_watts / (df_raw['CP_(Ext)'])) > 0, np.round((df_raw.cum_average_watts / (df_raw['CP_(Ext)']))*100, 0), 0)
df_raw['CP_scaled_cum_total_watts'] = np.where((df_raw.cum_total_watts / (df_raw['CP_(Ext)'])) > 0, np.round((df_raw.cum_total_watts / (df_raw['CP_(Ext)']))*100, 0), 0)

columns_cum = ['CP_scaled_cum_average_watts', 'CP_scaled_cum_total_watts']

for col in columns_cum:
    df_raw['log_' + col] = np.log1p(df_raw[col])
    df_raw['sqrt_' + col] = np.sqrt(df_raw[col])
    df_raw['exp_' + col] = df_raw[col].apply(lambda x: x**(2))
    df_raw['cub_' + col] = df_raw[col].apply(lambda x: x**(3))

df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 3076.78 MB
Decreased by 4.6%


In [35]:
# add scaled wattage variables based on CP ext

df_raw['xPower_EMA'] = df_raw['xPower_EMA'].astype(int)
df_raw['NP_EMA'] = df_raw['NP_EMA'].astype(int)

watts_cols = [x for x in df_raw.columns if x.endswith('sma_watts') 
    or x.endswith('lag_watts') or x.endswith('ewma_watts') or x.endswith('_EMA') or x =='Average_Power' or x =='Nonzero_Average_Power'] 

for col in watts_cols:
    new_col = 'CP_scaled_' + col
    df_raw[new_col] = np.where((df_raw[col] / (df_raw['CP_(Ext)'])) > 0, np.round((df_raw[col] / (df_raw['CP_(Ext)']))*100, 0), 0)

# reduce memory usage
df_raw = reduce_mem_usage(df_raw)

Memory usage after optimization is: 3386.39 MB
Decreased by 8.4%


After this last step we will save the file and use it in our next notebook **3. Feature exploration and selection.ipynb**

In [36]:
# Save the file as pickle file

df_raw.to_pickle('df_modelset_rider1_4.pkl')